In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Ensure all rows and columns are printed
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [9]:
# load in the mouse and human atlases

human_labels = sitk.ReadImage("./atlases/human_segmentation_mask.nii")
mouse_labels = sitk.ReadImage("./atlases/mouse_c57_fixed_labels_resized.nii")

human_labels_array = sitk.GetArrayFromImage(human_labels)
mouse_labels_array = sitk.GetArrayFromImage(mouse_labels)

print("human_labels_array dim:", human_labels_array.shape)
print("mouse_labels_array dim:", mouse_labels_array.shape)

human_labels_df = pd.read_csv("./atlases/human_CerebrA_LabelDetails.csv")
mouse_labels_df = pd.read_csv("./atlases/mouse_c57_brain_atlas_labels.csv")

# to aid with string matching convert all labels to lowercase
human_labels_df["Label Name"] = human_labels_df["Label Name"].astype(str).str.lower()

# Other overlapping labels include:

common_label_diff_names = { # "label_mouse": "label_human"
    "optic tract": "optic chiasm",
    "arbor vita of cerebellum": "cerebellum white matter",
    "nucleus accumbens": "accumbens area",
    "globus pallidus": "pallidum",
}


human_labels_array dim: (193, 229, 193)
mouse_labels_array dim: (282, 548, 420)


In [10]:
# relabel the regions with the same values in the mouse and human datasets

# Find common elements
common = list(set(human_labels_df["Label Name"]) & set(mouse_labels_df["Structure"]))

# Print results
print("Number of common labels:", len(common))
print("Common labels:", common)



# want to give these the same labels to display in seg masks!!
# just relabel them to be greater the max of both images
# and relabel the intensity in the image
max_int = int(np.maximum(np.max(human_labels_array), np.max(mouse_labels_array)))

# check that matches the max label in dfs
max_dfs = np.maximum(np.max(human_labels_df[["RH Label", "LH Labels"]]), np.max(mouse_labels_df[["right label", "left label"]]))

new_label_val = max_int + 1

## ISSUE: woith hjow the keys are being extracted
dict_keys = [i for i in common_label_diff_names.keys()]
common += dict_keys
print("Updated common labels:",common)


# now relabel starting at 251, 252 pair
for label in common:
    print("label:", label)
    if (label in common_label_diff_names):
        label_mouse = label
        label_human = common_label_diff_names[label]
    else:
        label_mouse = label
        label_human = label

    # change the intensities in the image
    # get current intesities to know what pxls to filter
    human_right_intensity = human_labels_df["RH Label"][human_labels_df["Label Name"] == label_human].values[0]
    human_left_intensity = human_labels_df["LH Labels"][human_labels_df["Label Name"] == label_human].values[0]

    mouse_right_intensity = mouse_labels_df["right label"][mouse_labels_df["Structure"] == label_mouse].values[0]
    mouse_left_intensity = mouse_labels_df["left label"][mouse_labels_df["Structure"] == label_mouse].values[0]

    # overwrite these intensity values with the new ones
    # overwritting values in a numpy array
    human_labels_array[human_labels_array == human_right_intensity] = new_label_val
    mouse_labels_array[mouse_labels_array == mouse_right_intensity] = new_label_val

    human_labels_array[human_labels_array == human_left_intensity] = new_label_val + 1
    mouse_labels_array[mouse_labels_array == mouse_left_intensity] = new_label_val + 1

    
    # relabel both df labels
    human_labels_df.loc[human_labels_df["Label Name"] == label_human, "RH Label"] = new_label_val
    mouse_labels_df.loc[mouse_labels_df["Structure"] == label_mouse, "right label"] = new_label_val
    new_label_val += 1

    human_labels_df.loc[human_labels_df["Label Name"] == label_human, "LH Labels"] = new_label_val
    mouse_labels_df.loc[mouse_labels_df["Structure"] == label_mouse, "left label"] = new_label_val

    new_label_val += 1



Number of common labels: 7
Common labels: ['amygdala', 'fourth ventricle', 'lateral ventricle', 'third ventricle', 'thalamus', 'hippocampus', 'basal forebrain']
Updated common labels: ['amygdala', 'fourth ventricle', 'lateral ventricle', 'third ventricle', 'thalamus', 'hippocampus', 'basal forebrain', 'optic tract', 'arbor vita of cerebellum', 'nucleus accumbens', 'globus pallidus']
label: amygdala
label: fourth ventricle
label: lateral ventricle
label: third ventricle
label: thalamus
label: hippocampus
label: basal forebrain
label: optic tract
label: arbor vita of cerebellum
label: nucleus accumbens
label: globus pallidus


In [11]:
# save the updated labeled atlases
mouse_labels_img = sitk.GetImageFromArray(mouse_labels_array)
human_labels_img = sitk.GetImageFromArray(human_labels_array)

# resample the smaller mask to the larger one
reference_image = mouse_labels_img # since larger
moving_image = human_labels_img # since smaller

# linear transform??
resampled_image = sitk.Resample(moving_image, reference_image)

human_labels_img = resampled_image

# Save result


sitk.WriteImage(mouse_labels_img, "./atlases/mouse_labels_common.nii")
sitk.WriteImage(human_labels_img, "./atlases/human_labels_common.nii")

mouse_labels_df.to_csv("./atlases/mouse_labels_common.csv", index=False)
human_labels_df.to_csv("./atlases/human_labels_common.csv", index=False)



In [12]:
human_labels_df

,Mindboggle ID,Label Name,RH Label,LH Labels,Notes,Dice Kappa
0,2002,caudal anterior cingulate,30,81,NaN,0.79
1,2003,caudal middle frontal,42,93,Improved distinction from Precentral,0.73
2,2005,cuneus,43,94,NaN,0.67
3,2006,entorhinal,36,87,Improved delimitation,0.78
4,2007,fusiform,24,75,NaN,0.77
5,2008,inferior parietal,10,61,NaN,0.75
6,2009,inferior temporal,3,54,Removed dorsal part MT,0.72
7,2010,isthmus cingulate,33,84,NaN,0.79
8,2011,lateral occipital,34,85,NaN,0.76
9,2012,lateral orbitofrontal,7,58,NaN,0.80


In [13]:
mouse_labels_df

,ID,Structure,right label,left label,hierarchy,tissue type,boundary
0,1,amygdala,251,252,NaN,WM,inside
1,2,anterior commissure: pars anterior,115,215,NaN,WM,inside
2,3,anterior commissure: pars posterior,23,103,NaN,WM,inside
3,4,arbor vita of cerebellum,267,268,NaN,WM,inside
4,5,basal forebrain,263,264,NaN,WM,inside
5,6,bed nucleus of stria terminalis,176,76,NaN,WM,inside
6,7,cerebellar cortex,202,2,NaN,WM,inside
7,8,cerebellar peduncle: inferior,123,223,NaN,WM,inside
8,9,cerebellar peduncle: middle,45,245,NaN,WM,inside
9,10,cerebellar peduncle: superior,242,222,NaN,WM,inside
